In [ ]:
!pip install plotly
!pip install pandas

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objects as go

# Table III

In [ ]:
def extract_mean_std(column):
    mean = [float(x.split('(')[0]) for x in column]
    std = [float(x.split('±')[1].rstrip(')')) for x in column]
    return mean, std

In [ ]:
def plot_with_shading(data,
                      title="",
                      yaxis_title = "Best Test Micro-F1",
                      base_color="red", our_color="blue",
                      base_name="base-model", our_name="our-model"):

    base_mean, base_std = extract_mean_std(data[base_name])
    our_mean, our_std = extract_mean_std(data[our_name])

    mean_df = pd.DataFrame({
        "b": data["b"],
        base_name: base_mean,
        our_name: our_mean
    })

    std_df = pd.DataFrame({
        "b": data["b"],
        base_name: base_std,
        our_name: our_std
    })

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=mean_df["b"],
        y=mean_df[base_name],
        mode="lines+markers",
        name="MultiBiSage",
        line=dict(color=base_color)
    ))

    fig.add_trace(go.Scatter(
        x=mean_df["b"].tolist() + mean_df["b"][::-1].tolist(),
        y=(mean_df[base_name] + std_df[base_name]).tolist() +
          (mean_df[base_name] - std_df[base_name])[::-1].tolist(),
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.1)' if base_color == "blue" else 'rgba(0, 0, 0, 0.1)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ))

    fig.add_trace(go.Scatter(
        x=mean_df["b"],
        y=mean_df[our_name],
        mode="lines+markers",
        name="BLB-HGNN(MB)",
        line=dict(color=our_color)
    ))

    fig.add_trace(go.Scatter(
        x=mean_df["b"].tolist() + mean_df["b"][::-1].tolist(),
        y=(mean_df[our_name] + std_df[our_name]).tolist() +
          (mean_df[our_name] - std_df[our_name])[::-1].tolist(),
        fill='toself',
        fillcolor='rgba(0, 128, 0, 0.1)' if our_color == "green" else 'rgba(255, 165, 0, 0.1)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ))

    fig.update_layout(
        title= {
            'text': title,
            'x': 0.5,
            'y': 0.85,
            'xanchor': 'center'
        },
        xaxis_title="b",
        yaxis_title=yaxis_title,
        xaxis=dict(
            tickmode='array',
            tickvals=[0.1, 0.2, 0.3, 0.4, 1.0],
            ticktext=["0.1", "0.2", "0.3", "0.4", "1.0"]
        ),
        template="simple_white",
        height=400,
        width=500,
        legend_traceorder="reversed",
        legend=dict(
          orientation="h",
          yanchor="bottom",
          y=1.02,
          xanchor="right",
          x=0.9
      )
    )

    fig.show()
    return fig


In [ ]:
data_iii = {
    "b": [0.1, 0.2, 0.3, 0.4, 1.0],
    "base-model": ["31.19(±0.84)", "35.40(±0.30)", "37.11(±1.21)", "38.47(±0.60)", "42.80(±0.28)"],
    "our-model": ["34.34(±0.18)", "38.30(±0.25)", "40.38(±0.29)", "41.50(±0.48)", "44.93(±0.28)"],
}

fig_iii = plot_with_shading(data_iii)

# Table IV

In [ ]:
data_iv = {
    "b": [0.1, 0.2, 0.3, 0.4, 1.0],
    "base-model": ["55.18(±0.31)", "58.89(±0.45)", "60.58(±0.22)", "61.81(±0.13)", "65.45(±0.16)"],
    "our-model": ["59.22(±0.21)", "60.58(±0.49)", "62.40(±0.31)", "63.53(±0.56)", "66.33(±0.23)"]
}

fig_iv = plot_with_shading(data_iv)

In [ ]:
from plotly.subplots import make_subplots

fig_grid = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Table III: Micro-F1", "Table IV: Micro-F1")
)

for i, trace in enumerate(fig_iii.data):
    trace.showlegend = i in [0, 2]
    fig_grid.add_trace(trace, row=1, col=1)

for i, trace in enumerate(fig_iv.data):
    trace.showlegend = False
    fig_grid.add_trace(trace, row=1, col=2)

fig_grid.update_layout(
    template="simple_white",
    showlegend=True,
    height=400,
    width=900
)

fig_grid.show()


# Table V

In [ ]:
data_v = pd.DataFrame({
    "b": [0.1, 0.2, 0.3, 0.4, 1.0],
    "base-model": [33.60, 38.12, 42.42, 44.50, 49.74],
    "our-model": [39.49, 41.96, 44.40, 47.35, 53.20]
})

data_v_2 = {
    "b": data_v["b"].tolist(),
    "base-model": [f"{x:.2f}(±0.01)" for x in data_v["base-model"]],
    "our-model": [f"{x:.2f}(±0.01)" for x in data_v["our-model"]],
}

fig_v = plot_with_shading(
    data_v_2,
    yaxis_title="Best Test Macro-F1",
    base_color="red",
    our_color="blue",
)

# Table VI

In [ ]:
def def_time(data,
             title="",
             base_name="base-model", our_name="our-model",
             base_color="red", our_color="blue",
             yaxis_title="Training Time (hours)"):

    base_mean, base_std = extract_mean_std(data[base_name])
    our_mean, our_std = extract_mean_std(data[our_name])

    mean_df = pd.DataFrame({
        "b": data["b"],
        base_name: base_mean,
        our_name: our_mean
    })

    std_df = pd.DataFrame({
        "b": data["b"],
        base_name: base_std,
        our_name: our_std
    })

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=mean_df["b"],
        y=mean_df[base_name],
        mode="lines+markers",
        name="MultiBiSage",
        line=dict(color=base_color, width=2, dash="dash")
    ))

    fig.add_trace(go.Scatter(
        x=mean_df["b"].tolist() + mean_df["b"][::-1].tolist(),
        y=(mean_df[base_name] + std_df[base_name]).tolist() +
          (mean_df[base_name] - std_df[base_name])[::-1].tolist(),
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.1)' if base_color == "blue" else 'rgba(0, 0, 0, 0.1)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ))

    fig.add_trace(go.Scatter(
        x=mean_df["b"],
        y=mean_df[our_name],
        mode="lines+markers",
        name="BLB-HGNN(MB)",
        line=dict(color=our_color, width=2)
    ))

    fig.add_trace(go.Scatter(
        x=mean_df["b"].tolist() + mean_df["b"][::-1].tolist(),
        y=(mean_df[our_name] + std_df[our_name]).tolist() +
          (mean_df[our_name] - std_df[our_name])[::-1].tolist(),
        fill='toself',
        fillcolor='rgba(255, 165, 0, 0.1)' if our_color == "orange" else 'rgba(0, 128, 0, 0.1)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ))

    # fig.update_layout(
    #     xaxis_title="b",
    #     yaxis_title=yaxis_title,
    #     template="simple_white"
    # )

    fig.update_layout(
        title={
            'text': title,
            'x': 0.5,
            'y': 0.85,
            'xanchor': 'center'
        },
        xaxis_title="b",
        yaxis_title=yaxis_title,
        xaxis=dict(
            tickmode='array',
            tickvals=[0.1, 0.2, 0.3, 0.4, 1.0],
            ticktext=["0.1", "0.2", "0.3", "0.4", "1.0"]
        ),
        template="simple_white",
        height=400,
        width=500,
        legend_traceorder="reversed",
        legend=dict(
          orientation="h",
          yanchor="bottom",
          y=1.02,
          xanchor="right",
          x=0.9
      )
    )

    fig.show()

In [ ]:
data_vi = {
    "b": [0.1, 0.2, 0.3, 0.4, 1.0],
    "base-model": ["1.11(±0.01)", "1.62(±0.01)", "2.13(±0.02)", "2.64(±0.03)", "5.89(±0.21)"],
    "our-model": ["1.12(±0.01)", "1.62(±0.01)", "2.13(±0.01)", "2.66(±0.01)", "5.94(±0.05)"]
}

fig_vi = def_time(data_vi)

# Table VII

In [ ]:
data_vii = {
    "b": [0.1, 0.2, 0.3, 0.4, 1.0],
    "base-model": ["7.58(±0.16)", "9.58(±0.12)", "11.64(±0.27)", "13.71(±0.08)", "27.56(±0.15)"],
    "our-model": ["7.05(±0.59)", "9.38(±0.03)", "11.52(±0.06)", "13.85(±0.34)", "25.97(±0.42)"]
}

fig_vii = def_time(data_vii)